In [1]:
# Libraries and Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
# from ANOTHER MACHINE LEARNING LIBRARY import SOMETHING ELSE

In [11]:
# APIs
# Use Alpaca for BTC data
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [12]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

# Inputs

In [13]:
# Read in Data Files
start_date = pd.Timestamp("2013-07-03", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-07-03", tz="America/New_York").isoformat()
tickers = ['BTC']
timeframe = "1Day"
df_stock = api.get_bars(
        tickers,
        timeframe,
        start=start_date,
        end=end_date
    ).df
df_stock.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-16 04:00:00+00:00,100.890,100.89,98.770,99.2447,4497,186,100.273227,BTC
2021-04-19 04:00:00+00:00,99.120,99.13,98.985,99.1012,4551,96,99.104105,BTC
2021-04-20 04:00:00+00:00,98.790,99.30,98.790,99.0441,3736,82,98.932931,BTC
2021-04-21 04:00:00+00:00,99.010,99.01,98.900,98.9705,997,42,98.950429,BTC
2021-04-22 04:00:00+00:00,98.925,99.04,98.860,99.0010,1599,68,98.958802,BTC


# Data Processing

# Machine Learning Analysis

# Save each of your alternative models as an HDF5 file

# Output
## Connect to an External Exchange (Demo Account)
or
## Send an SMS (Backup)

# Nice to have: Sentiment Analysis APIs